In [1]:
from sympy.printing.pytorch import torch
from torch.nn.functional import dropout

from model import *
from tokenizer import *

In [2]:
tokenizer = Tokenizer()

In [3]:
src_emb = InputEmbeddings(64, 65536)
src_pos = PositionalEncoding(64, 1000, 0.1)

In [4]:
tokens = tokenizer.encode("/Users/daniilogorodnikov/Documents/1706.03762v7.pdf")

In [5]:
emb = src_emb(torch.tensor(tokens[:1000]))

In [27]:
src = src_pos(emb.unsqueeze(0))

In [28]:
N = 6
d_model = 64
h = 8
dropout = 0.1
d_ff = 256

In [29]:
encoder_blocks = []
for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
    encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
    encoder_blocks.append(encoder_block)

In [30]:
encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))

In [35]:
mask = torch.triu(torch.ones((1, 1000, 1000)), diagonal=1).type(torch.int)

In [36]:
src.shape

torch.Size([1, 1000, 64])

In [40]:
encoder(src, mask).shape

torch.Size([1, 1000, 64])